In [ ]:
from collections import deque, defaultdict
from UserClasses import generateTree, generateList, ListNode, TreeNode
from math import sin, cos, pi
from functools import reduce
import re
import heapq

In [ ]:
#85. Maximal Rectangle
class MaximalRectangle(object):
    def maximalRectangle(self, matrix):

In [ ]:
#1140. Stone Game II
class StoneGameII(object):
    def _take_pile(self, prefix_sum, start_pos, to_take_amount, hand):
        res_hand = list(hand) 
        res_hand[0] += prefix_sum[start_pos+to_take_amount] - prefix_sum[start_pos]
        res_hand[1] = start_pos+to_take_amount
        res_hand[2] = to_take_amount
        return res_hand

    def _append_turn_res(self, prev_turn, hand_dict, prefix_sum, to_take_amount):
        if (hand_dict[prev_turn][1] + to_take_amount) >= len(prefix_sum):
            return None
        prev_hand = list(hand_dict[prev_turn])
        res_turn = 'A' if prev_turn == 'B' else 'B'
        res_hand = self._take_pile(prefix_sum, prev_hand[1], to_take_amount, hand_dict[res_turn])
        return (res_turn, {prev_turn: prev_hand, res_turn: res_hand})
        
    def stoneGameII(self, piles):
        prefix_sum = [piles[0]]
        for pile in piles[1:]:
            prefix_sum.append(pile + prefix_sum[-1])
        dfs_stack = deque([])
        dfs_stack.append(('A', {'A': [prefix_sum[0], 0, 1], 'B': [0, 0, 1]}))
        dfs_stack.append(('A', {'A': [prefix_sum[1], 1, 2], 'B': [0, 0, 1]}))
        result = {'A': [0], 'B': [prefix_sum[-1]]}
        while dfs_stack:
            prev_turn, hand_dict = dfs_stack.pop()
            if hand_dict[prev_turn][1] == len(prefix_sum) - 1:
                pass
            else:
                for to_take_amount in range(1, 2 * hand_dict[prev_turn][2] + 1):
                    new_turn_res = self._append_turn_res(prev_turn, hand_dict, prefix_sum, to_take_amount)
                    if new_turn_res:
                        dfs_stack.append(new_turn_res)
                        
        return result

In [ ]:
#241. Different Ways to Add Parentheses
class DiffWaysToCompute(object):
    def _getNumOperation(self, pExpression):
        num_arr, oper_arr = [], ['']
        string = ''
        for index, char in enumerate(pExpression):
            if char not in ['*', '+', '-']:
                string += char
            if (index == len(pExpression) - 1) or (char in ['*', '+', '-']):
                num_arr.append(int(string))
                oper_arr.append(char)
                string = ''
        return [(oper_arr[index], num_arr[index]) for index in range(len(num_arr))]

    def _makeOperation(self, pCurExpr, pOper, pNum):
        result = []
        temp_expr = list(pCurExpr[:-1])
        if pOper == '*':
            temp_expr.append(pCurExpr[-1] * pNum)
        if pOper == '+':
            temp_expr.append(pCurExpr[-1] + pNum)
        if pOper == '-':
            temp_expr.append(pCurExpr[-1] - pNum)
        result.append(temp_expr)
        temp_expr = list(pCurExpr)
        temp_expr.extend([pOper, pNum])
        result.append(temp_expr)
        return result
    
    def diffWaysToCompute(self, expression):
        num_operation = self._getNumOperation(expression)
        stack = deque([[num_operation[0][1]]])
        for oper, num in num_operation[1:-1]:
            temp_stack = deque([])
            while stack:
                cur_expr = stack.pop()
                temp_stack.extend(self._makeOperation(cur_expr, oper, num))   
            stack = temp_stack
        return stack

In [ ]:
#Не проходит 50 какой-то тест
#2493. Divide Nodes Into the Maximum Number of Groups
class MagnificentSets:
    def magnificentSets(self, n, edges):
        graph = {}
        for node_1, node_2 in edges:
            graph.setdefault(node_1, set()).add(node_2)
            graph.setdefault(node_2, set()).add(node_1)

        result = 0
        for start_node in range(1, n+1):
            queue = deque([start_node])
            unchecked = {node for node in range(1, n+1)}
            temp_result = 0
            while unchecked:
                if not queue:
                    queue.append(unchecked.pop())
                    unchecked.add(queue[0])
                while queue:
                    next_nodes = set()
                    for _ in range(len(queue)):
                        cur_node = queue.popleft()
                        if cur_node not in unchecked:
                            return -1
                        unchecked.remove(cur_node)
                        for next_node in graph.get(cur_node, []):
                            if (next_node in unchecked) and (next_node not in next_nodes):
                                queue.append(next_node)
                                next_nodes.add(next_node)
                    temp_result += 1
            result = max(temp_result, result)
        return result

In [ ]:
#2467. Most Profitable Path in a Tree
class MostProfitablePath:
    def _buildGraph(self, edges):
        graph = {}
        for begin, end in edges:
            graph.setdefault(begin, set()).add(end)
            graph.setdefault(end, set()).add(begin)
        return graph

    def _findBobRoute(self, graph, start_node):
        stack = deque([(start_node, deque([start_node]))])
        while stack:
            cur_node, cur_route = stack.pop()
            if cur_node == 0:
                return cur_route
            for next_node in graph[cur_node]:
                next_route = deque(cur_route)
                next_route.append(next_node)
                stack.append((next_node, next_route))
                graph[next_node].remove(cur_node)
            graph.pop(cur_node)

    def _findAliceIncome(self, graph, amount, bob_route):
        queue = deque([(0, amount[0])])
        amount[bob_route.popleft()] = 0
        result = []
        while queue:
            next_bob_node = bob_route.popleft() if bob_route else 0
            for _ in range(len(queue)):
                cur_node, cur_income = queue.pop()
                if not graph[cur_node]:
                    result.append(cur_income)
                for next_node in graph[cur_node]:
                    next_income = cur_income + (amount[next_node] if next_node != next_bob_node
                                                                  else amount[next_node] // 2)
                    queue.append((next_node, next_income))
                    graph[next_node].remove(cur_node)
                graph.pop(cur_node)
            amount[next_bob_node] = 0
        return max(result)
    
    def mostProfitablePath(self, edges, bob, amount):
        graph = self._buildGraph(edges)
        bob_route = self._findBobRoute(graph, bob)
        graph = self._buildGraph(edges)
        return self._findAliceIncome(graph, amount, bob_route)

In [ ]:
#1092. Shortest Common Supersequence
class ShortestCommonSupersequence:
    def shortestCommonSupersequence(self, str1, str2):
        short, long = min(str1, str2, key=len), max(str1, str2, key=len)
        short_letters = {}
        for index, letter in enumerate(short):
            short_letters.setdefault(letter, list()).append(index)
        subseq = deque([])
        include_letters = set()
        for letter in long:
            for _ in range(len(subseq)):
                next_subseq = subseq.popleft()
                if (next_subseq[-1]+1 < len(short)) and (short[next_subseq[-1]+1] == letter):
                    next_subseq.append(next_subseq[-1]+1)
                include_letters.add(next_subseq[-1])
                subseq.append(next_subseq)
            for letter_index in short_letters.setdefault(letter, list()):
                if letter_index not in include_letters:
                    subseq.append([letter_index])
                    include_letters.add(letter_index)
        max_subseq = max(subseq, key=len)
        return len(short), set(num for l in subseq for num in l )

In [ ]:
#30. Substring with Concatenation of All Words
class FindSubstring:
    def findSubstring(self, s, words):
        words_index, words_amount = {}, {-1: 0}
        for word in words:
            words_index[word] = words_index.setdefault(word, len(words_index))
            words_amount[words_index[word]] = words_amount.setdefault(words_index[word], 0) + 1
        prefix = []
        word_length = len(words[0])
        right = word_length
        while right <= len(s):
            prefix.append(words_index.get(s[right-word_length:right], -1))
            right += 1
        result = []
        not_zero = set(words_index.values())
        for start in range(word_length):
            left, right = start, start
            cur_words_amount = dict(words_amount)
            for _ in range(len(words)):
                cur_words_amount[prefix[right]] -= 1
                if cur_words_amount[prefix[right]] == 0:
                    not_zero.remove(prefix[right])
                else:
                    not_zero.add(prefix[right])
                right += word_length
            right -= word_length
            while right < len(prefix):
                print(s[left:right+word_length])
                if not not_zero:
                    result.append(left)
                cur_words_amount[prefix[left]] += 1
                if cur_words_amount[prefix[left]] == 0:
                    not_zero.remove(prefix[left])
                else:
                    not_zero.add(prefix[left])
                left += word_length
                right += word_length
                if right < len(prefix):
                    cur_words_amount[prefix[right]] -= 1
                    if cur_words_amount[prefix[right]] == 0:
                        not_zero.remove(prefix[right])
                    else:
                        not_zero.add(prefix[right])
        return result

In [ ]:
#3108. Minimum Cost Walk in Weighted Graph

In [ ]:
#2999. Count the Number of Powerful Integers

In [ ]:
#3373. Maximize The Number Of Target Nodes After Connecting Trees-II
class MaxTargetNodes(object):
    def _buildGraph(self, edges):
        graph = {}
        for begin, end in edges:
            graph.setdefault(begin, set()).add(end)
            graph.setdefault(end, set()).add(begin)
        return graph

    def _findNodeDegree(self, graph, node):
        queue = deque([node])
        result = 0
        distance = 0
        visited = set()
        while queue:
            for _ in range(len(queue)):
                cur_node = queue.popleft()
                visited.add(cur_node)
                result += (distance % 2 == 0)
                for next_node in graph[cur_node]:
                    if next_node not in visited:
                        queue.append(next_node)
            distance += 1
        return result
        
    def maxTargetNodes(self, edges1, edges2):
        graph1 = self._buildGraph(edges1)
        graph2 = self._buildGraph(edges2)
        max_degree_2 = max([self._findNodeDegree(graph2, node) for node in graph2])
        result = []
        for node in graph1:
            result.append(self._findNodeDegree(graph1, node)+max_degree_2)
        return result